### simulation - with transaction fee, taxes / without dividends

# 초기 조건 / 데이터 임포트 / 벤치마크 계산

## 라이브러리 임포트 / 초기 조건 설정

In [142]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [143]:
import pandas as pd
import numpy as np
from datetime import datetime
import math

In [144]:
transaction_fee_rate = 0.0025
tax_rate = 0.22
tax_threshold = 2500000 / 1100
initial_cash = 10000

In [145]:
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

In [146]:
strategies = []
returns = []

## 데이터 임포트

In [147]:
df = pd.read_csv('/content/drive/MyDrive/quant_projects/momentum_strategies/SPY.csv')[:-2] # 2021-09까지 결과 출력
closes = df.Close.to_numpy()
opens = df.Open.to_numpy()

In [148]:
tb3_df = pd.read_csv('/content/drive/MyDrive/quant_projects/momentum_strategies/TB3MS.csv')
risk_free_rates = tb3_df.TB3MS.to_numpy() / 100

## mdd / statistics

In [149]:
# 출처: http://blog.quantylab.com/mdd.html
def get_mdd(x):
    """
    MDD(Maximum Draw-Down)
    :return: (peak_upper, peak_lower, mdd rate)
    """
    arr_v = np.array(x)
    peak_lower = np.argmax(np.maximum.accumulate(arr_v) - arr_v)
    if peak_lower == 0:
      peak_upper = 0
    else:
      peak_upper = np.argmax(arr_v[:peak_lower])
    return peak_upper, peak_lower, (arr_v[peak_lower] - arr_v[peak_upper]) / arr_v[peak_upper]

In [150]:
# highs = df.High.to_numpy()
# lows = df.Low.to_numpy()

In [151]:
# np.argmax(np.maximum.accumulate(highs))

In [152]:
# 월중 mdd 
# max((np.maximum.accumulate(highs) - lows) / np.maximum.accumulate(highs))

In [153]:
# 월중 mdd
# peak_lower = np.argmax((np.maximum.accumulate(highs) - lows) / np.maximum.accumulate(highs))
# peak_upper = np.argmax(highs[:peak_lower])
# print(peak_upper, peak_lower, (lows[peak_lower] - highs[peak_upper]) / highs[peak_upper])

In [154]:
# get_mdd(benchmark)

In [155]:
def annualize(closes, opens, month_lag):
  annual_returns = []
  for i in range(len(closes) // 12 +1):
    if (i+1)*12 > len(closes):
      annual_returns.append(round((closes[-1] - opens[i*12-month_lag]) / opens[i*12-month_lag],4))
    elif i*12-month_lag < 0:
      annual_returns.append(round((closes[i*12+11-month_lag] - opens[0]) / opens[0], 4))
    else:
      annual_returns.append(round((closes[i*12+11-month_lag] - opens[i*12-month_lag]) / opens[i*12-month_lag],4))
  return annual_returns

In [156]:
def statistics(name, closes, annual_returns, current_flag=False):
  mean_risk_free_rate = np.mean(risk_free_rates) if current_flag else 0.005
  cagr = round(pow(closes[-1] / closes[0], 1/ (round(len(closes) / 12))) - 1, 4)
  mean_annual = round(np.mean(annual_returns),4)
  stddev = round(np.std(annual_returns),4)
  sharpe = round((np.mean(annual_returns)- mean_risk_free_rate) / np.std(annual_returns),4)
  worst_year = round(min(annual_returns), 4)
  best_year = round(max(annual_returns), 4)
  mdd = round(get_mdd(closes)[2], 4)
  return {'name':name, 'initial_balance': round(closes[0], 2), 'final_balance':round(closes[-1],2), 'cagr':cagr, 'mean_annual':mean_annual, 'stddev':stddev, 'sharpe':sharpe, 'worst_year':worst_year, 'best_year': best_year, 'mdd': mdd}

## 벤치마크 계산

In [157]:
benchmark_shares = initial_cash // closes[0]
benchmark = closes * benchmark_shares

In [158]:
strategies.append({**statistics('Benchmark', benchmark, annualize(closes, opens, 1)), 'accum_fees':0, 'accum_taxes':0})
returns.append({"name":'Benchmark', "df":benchmark})

In [159]:
strategies

[{'accum_fees': 0,
  'accum_taxes': 0,
  'best_year': 0.3453,
  'cagr': 0.0814,
  'final_balance': 96556.5,
  'initial_balance': 9991.41,
  'mdd': -0.522,
  'mean_annual': 0.0922,
  'name': 'Benchmark',
  'sharpe': 0.5243,
  'stddev': 0.1664,
  'worst_year': -0.3842}]

# Simulation

In [43]:
# 세금/수수료 계산 테스트
transaction_fee_rate = 0
tax_rate = 0
start_lag = 2
rebalancing_period = 1
look_back = 12
for index in [0]:
  # 10to4이니까 2+2=4, 2+2+6=10
  df_temp = pd.DataFrame({'Date': df['Date'], 'Close': df['Close'], 'Open':df['Open']})
  simulation_detail = f'{index + start_lag}to{index + start_lag + rebalancing_period}'
  name = f'SPY_simulation_{rebalancing_period}:{look_back}_{simulation_detail}_test'
  cash, month_closes, month_opens = [np.array([initial_cash] * len(closes), dtype=np.float64) for _ in range(3)]
  upwards, positions, shares, annual_taxes, accum_taxes, month_profits, annual_profits, buy_prices, transaction_fees, accum_transaction_fees = [np.zeros(len(closes)) for _ in range(10)]
  for i in range(look_back, len(closes)):
    #조건에 만족하면 배당수익 기록 (보유 현금 증가로)
    # cash[i] = cash[i - rebalancing_period] dividends[i] * shares[i - rebalancing_period]
    if i % rebalancing_period == index:
      upwards[i] = 1 if (closes[i] - closes[i-look_back]) / closes[i-look_back] >= risk_free_rates[i] else 0
      month_opens[i] = cash[i-1] + shares[i-1] * opens[i]
      # if-else문에 따라 cash, positions, buy_prices, annual_profits 4개 항목 처리
      if upwards[i - rebalancing_period] == 0 and upwards[i] == 1:
        # 매수
        shares[i] = cash[i - 1] / (1 + transaction_fee_rate) // closes[i]
        positions[i] = shares[i] * closes[i]
        transaction_fees[i] = positions[i] * transaction_fee_rate
        cash[i] = cash[i-1] - positions[i] - transaction_fees[i]
        buy_prices[i] = closes[i]
      elif upwards[i - rebalancing_period] == 1 and upwards[i] == 0:
        # 매도
        position_size = shares[i-1] * closes[i]
        profit_size = shares[i-1] * (closes[i] - buy_prices[i-1])
        month_profits[i] = profit_size - position_size * transaction_fee_rate
        cash[i] = cash[i-1] + position_size * (1 - transaction_fee_rate)
        transaction_fees[i] = position_size * transaction_fee_rate
      else:
        # 둘다 현금보유 or 둘다 주식 보유
        cash[i] = cash[i - 1] #만약 지난달에 배당을 받았다면 주식:현금 비율이 변화하므로 리밸런싱 필요
        positions[i] = shares[i-1] * closes[i]
        shares[i] = shares[i-1]
        buy_prices[i] = buy_prices[i-1]
    else:
      month_opens[i] = cash[i-1] + shares[i-1] * opens[i]
      cash[i] = cash[i-1]
      shares[i] = shares[i-1]
      positions[i] = shares[i] * closes[i]
      buy_prices[i] = buy_prices[i-1]
    month_closes[i] = cash[i] + positions[i]

    # 연간 수익 계산
    if (i + start_lag) % 12 == 1:
      # 매년 1월 시점에서 연간 수익 == 1월 월간 수익, 매년 1월 연간 세금 초기화
      annual_profits[i] = month_profits[i]
    else:
      annual_profits[i] = annual_profits[i-1] + month_profits[i]

    # 연말(12월) 세금 계산
    if (i + start_lag) % 12 == 0:
      if annual_profits[i] > tax_threshold:
        annual_taxes[i] = (annual_profits[i] - tax_threshold) * tax_rate
        if cash[i] >= annual_taxes[i]:
          cash[i] -= annual_taxes[i]
        else:
          shares_minus = (annual_taxes[i] - cash[i]) * (1 + transaction_fee_rate) // closes[i] + 1
          shares[i] -= shares_minus
          positions[i] -= shares_minus * closes[i]
          transaction_fees[i] += shares_minus * closes[i] * transaction_fee_rate
          month_profits[i] += shares_minus * (closes[i] - buy_prices[i])
          annual_profits[i] += shares_minus * (closes[i] - buy_prices[i])
          if (annual_profits[i] > tax_threshold):
            annual_taxes[i] = (annual_profits[i] - tax_threshold) * tax_rate
            cash[i] += (shares_minus) * closes[i] - annual_taxes[i] - transaction_fees[i] 
          else:
            cash[i] += (shares_minus) * closes[i] - transaction_fees[i] 
      accum_taxes[i] = accum_taxes[i-1] + annual_taxes[i]
    else:
      accum_taxes[i] = accum_taxes[i-1]
    accum_transaction_fees[i] = accum_transaction_fees[i-1] + transaction_fees[i]
  df_temp['upwards'] = upwards.tolist()
  df_temp['shares'] = shares.tolist()
  df_temp['positions'] = positions.tolist()
  df_temp['cash'] = cash.tolist()
  df_temp['month_closes'] = month_closes.tolist()
  df_temp['month_opens'] = month_opens.tolist()
  df_temp['buy_prices'] = buy_prices.tolist()
  df_temp['transaction_fees'] = transaction_fees.tolist()
  df_temp['month_profits'] = month_profits.tolist()
  df_temp['annual_profits'] = annual_profits.tolist()
  df_temp['annual_taxes'] = annual_taxes.tolist()
  df_temp['benchmark'] = benchmark.tolist()
  df_temp.to_csv(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_{name}.csv')
  strategies.append({**statistics(name, month_closes, annualize(month_closes, month_opens, 1)), 'accum_fees':accum_transaction_fees[-1], 'accum_taxes':accum_taxes[-1]})
  returns.append({"name":name, "df":df_temp['month_closes'].to_numpy()})

In [160]:
start_lag = 2 # data starts from 1993-02
rebalancing_periods = [1,3, 6,12]
look_backs = [1,3,6,12]
for rebalancing_period in rebalancing_periods:
  for look_back in look_backs:
    for index in range(rebalancing_period):
      # 2개월 리밸런싱 -> 홀수달/짝수달, 3개월 리밸런싱 -> 0/1/2, 4개월 리밸런싱 -> 0/1/2/3
      for exclude_period in range(min(3, look_back)):
        df_temp = pd.DataFrame({'Date': df['Date'], 'Close': df['Close'], 'Open': df['Open']})
        simulation_detail = f'{index + start_lag}to{index + start_lag + rebalancing_period}ex{exclude_period}'
        name = f'SPY_simulation_{rebalancing_period}:{look_back}_{simulation_detail}_tax_fee'
        cash, month_closes, month_opens = [np.array([initial_cash] * len(closes), dtype=np.float64) for _ in range(3)]
        upwards, positions, shares, annual_taxes, accum_taxes, month_profits, annual_profits, buy_prices, transaction_fees, accum_transaction_fees = [np.zeros(len(closes)) for _ in range(10)]
        for i in range(look_back, len(closes)):
          #조건에 만족하면 배당수익 기록 (보유 현금 증가로)
          # cash[i] = cash[i - rebalancing_period] dividends[i] * shares[i - rebalancing_period]
          if i % rebalancing_period == index:
            upwards[i] = 1 if (closes[i-exclude_period] - closes[i-look_back]) / closes[i-look_back] >= risk_free_rates[i] else 0
            month_opens[i] = cash[i-1] + shares[i-1] * opens[i]
            # if-else문에 따라 cash, positions, buy_prices, annual_profits 4개 항목 처리
            if upwards[i - rebalancing_period] == 0 and upwards[i] == 1:
              # 매수
              shares[i] = cash[i - 1] / (1 + transaction_fee_rate) // closes[i]
              positions[i] = shares[i] * closes[i]
              transaction_fees[i] = positions[i] * transaction_fee_rate
              cash[i] = cash[i-1] - positions[i] - transaction_fees[i]
              buy_prices[i] = closes[i]
            elif upwards[i - rebalancing_period] == 1 and upwards[i] == 0:
              # 매도
              position_size = shares[i-1] * closes[i]
              profit_size = shares[i-1] * (closes[i] - buy_prices[i-1])
              month_profits[i] = profit_size - position_size * transaction_fee_rate
              cash[i] = cash[i-1] + position_size * (1 - transaction_fee_rate)
              transaction_fees[i] = position_size * transaction_fee_rate
            else:
              # 둘다 현금보유 or 둘다 주식 보유
              cash[i] = cash[i - 1] #만약 지난달에 배당을 받았다면 주식:현금 비율이 변화하므로 리밸런싱 필요
              positions[i] = shares[i-1] * closes[i]
              shares[i] = shares[i-1]
              buy_prices[i] = buy_prices[i-1]
          else:
            month_opens[i] = cash[i-1] + shares[i-1] * opens[i]
            cash[i] = cash[i-1]
            shares[i] = shares[i-1]
            positions[i] = shares[i] * closes[i]
            buy_prices[i] = buy_prices[i-1]
          month_closes[i] = cash[i] + positions[i]

          # 연간 수익 계산
          if (i + start_lag) % 12 == 1:
            # 매년 1월 시점에서 연간 수익 == 1월 월간 수익, 매년 1월 연간 세금 초기화
            annual_profits[i] = month_profits[i]
          else:
            annual_profits[i] = annual_profits[i-1] + month_profits[i]

          # 연말(12월) 세금 계산
          if (i + start_lag) % 12 == 0:
            if annual_profits[i] > tax_threshold:
              annual_taxes[i] = (annual_profits[i] - tax_threshold) * tax_rate
              if cash[i] >= annual_taxes[i]:
                cash[i] -= annual_taxes[i]
              else:
                shares_minus = (annual_taxes[i] - cash[i]) * (1 + transaction_fee_rate) // closes[i] + 1
                shares[i] -= shares_minus
                positions[i] -= shares_minus * closes[i]
                transaction_fees[i] += shares_minus * closes[i] * transaction_fee_rate
                month_profits[i] += shares_minus * (closes[i] - buy_prices[i])
                annual_profits[i] += shares_minus * (closes[i] - buy_prices[i])
                if (annual_profits[i] > tax_threshold):
                  annual_taxes[i] = (annual_profits[i] - tax_threshold) * tax_rate
                  cash[i] += (shares_minus) * closes[i] - annual_taxes[i] - transaction_fees[i] 
                else:
                  cash[i] += (shares_minus) * closes[i] - transaction_fees[i] 
            accum_taxes[i] = accum_taxes[i-1] + annual_taxes[i]
          else:
            accum_taxes[i] = accum_taxes[i-1]
          accum_transaction_fees[i] = accum_transaction_fees[i-1] + transaction_fees[i]
        df_temp['upwards'] = upwards.tolist()
        df_temp['shares'] = shares.tolist()
        df_temp['positions'] = positions.tolist()
        df_temp['cash'] = cash.tolist()
        df_temp['month_closes'] = month_closes.tolist()
        df_temp['month_opens'] = month_opens.tolist()
        df_temp['buy_prices'] = buy_prices.tolist()
        df_temp['transaction_fees'] = transaction_fees.tolist()
        df_temp['month_profits'] = month_profits.tolist()
        df_temp['annual_profits'] = annual_profits.tolist()
        df_temp['annual_taxes'] = annual_taxes.tolist()
        df_temp['benchmark'] = benchmark.tolist()
        # df_temp.to_csv(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_{name}.csv')
        strategies.append({**statistics(name, month_closes, annualize(month_closes, month_opens, 1)), 'accum_fees':accum_transaction_fees[-1], 'accum_taxes':accum_taxes[-1]})
        returns.append({"name":name, "df":df_temp['month_closes'].to_numpy()})

In [ ]:
strategies

# 결과 출력

## csv / jpg 출력

In [161]:
statistics_df = pd.DataFrame(strategies)
# print(statistics_df)
# print(statistics_df.iloc[statistics_df['final_balance'].idxmax()])

In [162]:
statistics_df.sort_values(by=['cagr'], inplace=True, ascending=False)

In [163]:
statistics_df

,name,initial_balance,final_balance,cagr,mean_annual,stddev,sharpe,worst_year,best_year,mdd,accum_fees,accum_taxes
91,SPY_simulation_6:12_4to10ex2_tax_fee,10000.00,101763.17,0.0833,0.0958,0.1113,0.8157,-0.1062,0.3082,-0.1990,753.744467,10272.048727
0,Benchmark,9991.41,96556.50,0.0814,0.0922,0.1664,0.5243,-0.3842,0.3453,-0.5220,0.000000,0.000000
193,SPY_simulation_12:12_4to16ex2_tax_fee,10000.00,94549.05,0.0805,0.0940,0.1170,0.7609,-0.0667,0.3450,-0.1988,617.324141,10155.309480
197,SPY_simulation_12:12_6to18ex0_tax_fee,10000.00,92750.46,0.0798,0.0878,0.1216,0.6810,-0.1284,0.3082,-0.1990,242.817285,2238.647683
89,SPY_simulation_6:12_4to10ex0_tax_fee,10000.00,92433.16,0.0797,0.0938,0.1077,0.8243,-0.0668,0.3080,-0.1989,1006.285430,14611.692710
...,...,...,...,...,...,...,...,...,...,...,...,...
102,SPY_simulation_12:1_3to15ex0_tax_fee,10000.00,15788.83,0.0159,0.0218,0.1092,0.1539,-0.2303,0.3948,-0.4537,274.714697,740.438939
142,SPY_simulation_12:3_11to23ex2_tax_fee,10000.00,14664.15,0.0133,0.0197,0.1086,0.1352,-0.3852,0.2820,-0.4350,364.205985,598.125946
4,SPY_simulation_1:3_2to3ex2_tax_fee,10000.00,14337.27,0.0125,0.0134,0.0670,0.1248,-0.1241,0.1559,-0.2648,3545.524205,0.000000
41,SPY_simulation_6:1_2to8ex0_tax_fee,10000.00,12019.74,0.0064,0.0082,0.0753,0.0419,-0.1397,0.1819,-0.3303,461.112652,0.000000


In [164]:
statistics_df.to_csv(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_statistics.csv')

In [165]:
returns_df = pd.DataFrame({'Date': df['Date']})
for portfolio in returns:
  returns_df[portfolio['name']] = portfolio['df']
print(returns_df)
returns_df.to_csv(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_returns.csv')

           Date  ...  SPY_simulation_12:12_13to25ex2_tax_fee
0    1993-02-01  ...                            10000.000000
1    1993-03-01  ...                            10000.000000
2    1993-04-01  ...                            10000.000000
3    1993-05-01  ...                            10000.000000
4    1993-06-01  ...                            10000.000000
..          ...  ...                                     ...
339  2021-05-01  ...                            36210.777954
340  2021-06-01  ...                            36900.497008
341  2021-07-01  ...                            37799.198040
342  2021-08-01  ...                            38921.497008
343  2021-09-01  ...                            36993.378470

[344 rows x 222 columns]


In [ ]:
chart = returns_df.plot.line(x='Date', grid=True, figsize=(40,25), xticks=np.arange(0, 346, 12), yticks=np.arange(0, 150000, 10000))
chart.tick_params(axis='x', rotation=90)
chart.figure.savefig(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_returns.jpg')